<a href="https://colab.research.google.com/github/RicoStaedeli/NLP2025_CQG/blob/main/2_baseline_cqs_generation_schema.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline Predictions
In this file we generate the baseline predictions

## Setup

In [1]:
!pip install -U transformers
!pip install accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [2]:
import torch
from google.colab import userdata, drive
import logging
import transformers
import os

In [3]:
token = userdata.get('GITHUB')
repo_url = f"https://{token}@github.com/RicoStaedeli/NLP2025_CQG.git"

!git clone {repo_url}

Cloning into 'NLP2025_CQG'...
remote: Enumerating objects: 648, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 648 (delta 87), reused 77 (delta 49), pack-reused 514 (from 1)
Receiving objects: 100% (648/648), 24.83 MiB | 25.87 MiB/s, done.
Resolving deltas: 100% (313/313), done.


In [4]:
os.chdir("NLP2025_CQG")
!ls

1_Preprocessing.ipynb			Development
2a_Baseline_Evaluation.ipynb		Doc
2_Baseline_CQS_generation.ipynb		Evaluation
2_Baseline_CQS_generation_old.ipynb	Evaluation_Schema.ipynb
2_baseline_cqs_generation_schema.ipynb	INFORMATION.md
3a_Finetuned_CQS_generation.ipynb	LICENSE
3b_Finetune_Evaluation.ipynb		Logs
3_Training.ipynb			README.md
4a_RAG_CQS_generation.ipynb		requirements.txt
4b_RAG_Evaluation.ipynb			SocratiQ_final_prepro.ipynb
4_RAG_System,.ipynb			Training
5_Evaluation_Analytics.ipynb		Utils
Data


In [5]:
################################################################################
#######################   PATH VARIABLES        ################################
################################################################################

model_name= "Meta-Llama-3.1-8B-Instruct-bnb-4bit"
model_id= f"unsloth/{model_name}"

test_dataset_path = f"Data/Processed/test.csv"

results_path = os.path.join(os.getcwd(), f"Evaluation/Results/results_schema_Baseline_{model_name}.json")

log_base_path = f"Logs/"
os.makedirs(log_base_path, exist_ok=True)

log_path = log_base_path + "2_baseline_generation.log"


################################################################################
#######################   STATIC VARIABLES      ################################
################################################################################


In [6]:
# Setup logger manually
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create file handler (only if not already added)
if not logger.handlers:
    fh = logging.FileHandler(log_path)
    fh.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    logger.addHandler(fh)

# Detect device
device = torch.device(
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)

# Log the device info
logger.info("--------  Start with Baseline Generation  -------------")
logger.info(f'Device selected: {device}')
logger.info(f'Results Path: {results_path}')
logger.info(f'Log Path: {log_path}')
logger.info("--------------------------------------------------------")

INFO:__main__:--------  Start with Baseline Generation  -------------
INFO:__main__:Device selected: cuda
INFO:__main__:Results Path: /content/NLP2025_CQG/Evaluation/Results/results_schema_Baseline_Meta-Llama-3.1-8B-Instruct-bnb-4bit.json
INFO:__main__:Log Path: Logs/2_baseline_generation.log
INFO:__main__:--------------------------------------------------------


## Generate Answers

In [7]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [33]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="cuda",
    quantization_config={"load_in_4bit": True},
    low_cpu_mem_usage=True,
)

def generate_response(prompt_texts, max_new_tokens=64):
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id,
    )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    responses = []
    for full_output in decoded:
        # Extract text after ### Response:
        if "### Response:" in full_output:
            response = full_output.split("### Response:")[-1].strip()
        else:
            response = full_output.strip()

        # Remove leading 'assistant\n\n' if present
        if response.lower().startswith("assistant"):
            response = response[len("assistant"):].lstrip("\n ").strip()

        responses.append(response)
    return responses

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


In [32]:
import gc

del model
gc.collect()
torch.cuda.empty_cache()

In [9]:
schemas = [
    """'Cause to Effect' with the examples:
    How strong is the generalisation that if <eventA> then <eventB>?
    Are there other factors in this particular case that could have interfered with the event of‘<eventB>’?
    """,
    """ 'Expert Opinion' with the examples:
    Is <expertE> a genuine expert in <domainD>?
    Did <expertE> really assert that <eventA>? Is <expertE>’s pronouncement directly quoted? If not, is a reference to the original source given? Can it be checked?
    If <expertE>’s advice is not quoted, does it look like important information or qualifications may have been left out?
    Is what <expertE> said clear? Are there technical terms used that are not explained clearly?
    Is <eventA> relevant to domain <domainD>?
    Is <eventA> consistent with what other experts in <domainD> say?
    Is <eventA> consistent with known evidence in <domainD>?
    """,
    """'Analogy' with the examples:
    Are <C1> and <C2> similar in the respect cited?
    Is <eventA> true in <C1>?
    Are there differences between <C1> and <C2> that would tend to undermine the force of the similarity cited?
    Is there some other case that is also similar to <C1>, but in which <eventA> is false?
    """,
    """'Fear Appeal' with the examples:
    Is <eventB> bad? Why and to whom is it bad?
    Is <eventA> away to prevent <eventB>?
    Is it practically possible for <eventA> to happen?
    Are there other consequences from <eventA>?
    """
    ]

In [10]:
!pip install tqdm
from tqdm.auto import tqdm

In [49]:
import pandas as pd
import json
from collections import defaultdict
results = {}
chunk_size = 10

for chunk in pd.read_csv(test_dataset_path, chunksize=chunk_size):
    contexts = chunk['input'].tolist()
    ids = chunk['id'].tolist()
    prompts = []
    input_ids = []

    for idx, input_text in enumerate(contexts):
        input_id = ids[idx]
        for schema in schemas:
            messages = [
                {"role": "system", "content": "You are a system designed to generate critical questions for a given argumentative context."},
                {"role": "user", "content": f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate one critical question addressing the given context following the schema:

### Schema:
{schema}

Your answer is just the question without anything else.

This is the given context to relate the question to:

### Context:
{input_text}

### Response:
"""}
            ]
            prompt_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            prompts.append(prompt_text)
            input_ids.append(input_id)

    batch_size = 64
    num_batches = (len(prompts) + batch_size - 1) // batch_size

    with tqdm(total=num_batches, desc="Generating Critical Questions", leave=True) as pbar:
        for batch_start in range(0, len(prompts), batch_size):
            batch_prompts = prompts[batch_start:batch_start+batch_size]
            batch_outputs = generate_response(batch_prompts, max_new_tokens=256)

            for curr_id, output in zip(input_ids[batch_start:batch_start + batch_size], batch_outputs):
                if curr_id not in results:
                    results[curr_id] = {
                        "input": chunk.loc[chunk['id'] == curr_id, 'input'].values[0],
                        "cqs": []
                    }
                results[curr_id]['cqs'].append({"cq": output})

            torch.cuda.empty_cache()
            gc.collect()
            pbar.update(1)

with open(results_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"Results saved to {results_path}")


Generating Critical Questions:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Critical Questions:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Critical Questions:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Critical Questions:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Critical Questions:   0%|          | 0/1 [00:00<?, ?it/s]

...

Generating Critical Questions:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Critical Questions:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Critical Questions:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Critical Questions:   0%|          | 0/1 [00:00<?, ?it/s]

Results saved to /content/NLP2025_CQG/Evaluation/Results/results_schema_Baseline_Meta-Llama-3.1-8B-Instruct-bnb-4bit.json


## Commit & Push

In [50]:
!git config --global user.name "Showcas"
!git config --global user.email "cedric.bohni@gmx.de"


commit_message = f"Done baseline"
!git add .
!git commit -m "{commit_message}"
!git push

[main 9c9b5c5] Done baseline
 1 file changed, 3164 insertions(+), 2284 deletions(-)
 rewrite Evaluation/Results/results_schema_Baseline_Meta-Llama-3.1-8B-Instruct-bnb-4bit.json (80%)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 12 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 96.66 KiB | 4.60 MiB/s, done.
Total 5 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/RicoStaedeli/NLP2025_CQG.git
   de875f0..9c9b5c5  main -> main
